In [ ]:
# Get Data to play with
import pandas as pd
import numpy as np


### Linear Interpolation

### Cubic Interpolation

### Polynomial Interpolation

### Volumetric Interpolation

### Mean Imputation Strategy

### Median Imputation Strategy

### Last Observation Carried Forward / Forward Fill 

### Next Observation Carried Forward / Backward Fill

### Spline Interpolation